In [ ]:
import numpy as np
import pandas as pd

np.random.seed()
N = 1000  # super large number of samples

# --------------- Domain 1: Route Planning ---------------
route_planning = pd.DataFrame({
    'origin_x': np.random.uniform(0, 100, N),
    'origin_y': np.random.uniform(0, 100, N),
    'dest_x': np.random.uniform(0, 100, N),
    'dest_y': np.random.uniform(0, 100, N),
    'traffic_density': np.random.uniform(0, 1, N),
    'road_type': np.random.choice([1, 2, 3, 4, 5], N),
})
route_planning['distance'] = np.sqrt(
    (route_planning['dest_x'] - route_planning['origin_x'])**2 +
    (route_planning['dest_y'] - route_planning['origin_y'])**2
)
speed_base = {1:60, 2:50, 3:40, 4:30, 5:20}
route_planning['speed'] = route_planning['road_type'].map(speed_base) * np.random.uniform(0.7, 1.3, N)
route_planning['travel_time'] = (route_planning['distance']/route_planning['speed'])*60*\
                                (1 + route_planning['traffic_density']*np.random.uniform(0.05,0.7,N))

# --------------- Domain 2: Vehicle Assignment ---------------
vehicle_assignment = pd.DataFrame({
    'vehicle_capacity': np.random.randint(50,500,N),
    'battery_level': np.random.uniform(0.2,1.0,N),
    'delivery_size': np.random.randint(5,150,N),
    'vehicle_type': np.random.choice([1,2,3],N),
    'speed_factor': np.random.uniform(0.7,1.2,N),
})
vehicle_assignment['assigned_speed'] = route_planning['speed'] * vehicle_assignment['speed_factor']
vehicle_assignment['load_utilization'] = vehicle_assignment['delivery_size']/vehicle_assignment['vehicle_capacity']

# --------------- Domain 3: Time Scheduling ---------------
time_scheduling = pd.DataFrame({
    'requested_time': np.random.randint(0,24,N),
    'delivery_priority': np.random.randint(1,10,N),
    'customer_patience': np.random.uniform(0,1,N),
})
time_scheduling['delay_probability'] = np.clip(
    (route_planning['travel_time']/60)*(1+vehicle_assignment['load_utilization']*0.7)*np.random.uniform(0.5,1.5,N),
    0,1
)

# --------------- Domain 4: Dynamic Rerouting & Traffic ---------------
dynamic_rerouting = pd.DataFrame({
    'current_x': np.random.uniform(0,100,N),
    'current_y': np.random.uniform(0,100,N),
    'traffic_updates': np.random.uniform(0,1,N),
    'new_delivery_requests': np.random.randint(0,5,N),
    'vehicle_status': np.random.choice([0,1],N),
    'weather': np.random.choice([0,1,2],N),  # 0=clear,1=rain,2=storm
})
dynamic_rerouting['congestion_score'] = dynamic_rerouting['traffic_updates'] + \
                                       dynamic_rerouting['new_delivery_requests']*0.5 + \
                                       dynamic_rerouting['weather']*0.7 + \
                                       (route_planning['travel_time']/route_planning['travel_time'].max())*0.5

# --------------- Domain 5: Energy & Emissions ---------------
energy_emissions = pd.DataFrame({
    'fuel_level': np.random.uniform(0,1,N),
    'emission_rate': np.random.uniform(0,1,N),
    'energy_consumed': vehicle_assignment['delivery_size']*np.random.uniform(0.1,0.5,N),
    'regeneration_rate': np.random.uniform(0,0.2,N)
})
energy_emissions['net_energy'] = energy_emissions['fuel_level'] - energy_emissions['energy_consumed'] + energy_emissions['regeneration_rate']

# --------------- Combine and Normalize ---------------
datasets = [route_planning, vehicle_assignment, time_scheduling, dynamic_rerouting, energy_emissions]
DATA_MATRIX = np.hstack([df.values for df in datasets])
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0)+1e-8)

# --------------- Graph-based Multi-node Targets ---------------
D_graph = 5  # number of nodes=
candidate_dims =candidate_dims = [8, 6, 4, 3, 6]  # D_graph=5
 # each node has its own target dimension


# ---------------- Combine and normalize ----------------
datasets = [route_planning, vehicle_assignment, time_scheduling, dynamic_rerouting]
dataset_dims = [df.shape[1] for df in datasets]
max_dim = max(dataset_dims)
padded_data = [df.values for df in datasets]

DATA_MATRIX = np.hstack(padded_data)
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0)+1e-8)
def generate_targets_per_node(DATA_MATRIX, candidate_dims, D_graph):
    """
    Generate target vectors for each node with its own candidate dimension.

    Parameters:
    - DATA_MATRIX: normalized data array
    - candidate_dims: list of ints, length = D_graph, each specifying the target vector size per node
    - D_graph: number of graph nodes

    Returns:
    - List of dicts: each dict contains the node's target vector under key 'target'
    """
    targets = []
    for node_idx in range(D_graph):
        row = DATA_MATRIX[node_idx % len(DATA_MATRIX)]
        dim = candidate_dims[node_idx]  # get the dim for this node
        if len(row) >= dim:
            sampled = row[:dim]
        else:
            sampled = np.pad(row, (0, dim - len(row)), constant_values=0.5)
        targets.append({'target': sampled})
    return targets

# Example: each node has its own target dimension
candidate_dims = [8, 6, 4, 3,6]  # D_graph=5
D_graph = len(candidate_dims)

synthetic_targets = generate_targets_per_node(DATA_MATRIX, candidate_dims, D_graph)

# check
for i, t in enumerate(synthetic_targets):
    print(f"Node {i} target size: {len(t['target'])}")


# ACO Multiplex

In [ ]:
import numpy as np

FEATURE_KEYS = ['Distance', 'Speed', 'Load', 'Capacity', 'TempRequirement', 'Containerization']
FEATURE_TARGET = [[1]*len(FEATURE_KEYS) for _ in range(D_graph)]
METRIC_KEYS = ['TravelTime', 'FuelEfficiency', 'CargoHandling', 'OperationalDelay', 'EnergyConsumption']
METRIC_TARGET = [[1]*len(METRIC_KEYS) for _ in range(D_graph)]

class MetricsEvaluator:
    def __init__(self, data_matrix):
        self.data_matrix = data_matrix
        self.num_features = data_matrix.shape[1]
        self.num_nodes = data_matrix.shape[0]

        # Baseline parameters
        self.W0, self.T0, self.U0 = 15.0, 100.0, 10.0
        self.P0 = 10.0

        # ACO-like parameters
        self.aco_num_ants = 10        # increased ants for better exploration
        self.aco_iterations = 300      # more iterations for convergence
        self.evaporation = 0.1         # slightly higher to avoid pheromone saturation
        self.pheromone_init = 0.7

    def compute_node_metrics(self, node_idx, y=None, node_size=4):
        cols_per_node = self.num_features // self.num_nodes
        start_col = node_idx * cols_per_node
        node_cols = slice(start_col, start_col + node_size)
        node_data = self.data_matrix[:, node_cols]

        # Compute mean signals per node
        signals_full = node_data.mean(axis=0)
        signals = np.zeros(4)
        for i in range(min(4, len(signals_full))):
            signals[i] = signals_full[i]

        # Prepare y vector
        if y is None:
            y = np.zeros(3)
        else:
            y = np.array(y)
            if len(y) < 3:
                y = np.pad(y, (0, 3 - len(y)), constant_values=0.5)
            else:
                y = y[:3]

        # Extract features safely
        travel_time = node_data[0, 0] if node_data.shape[1] > 0 else 0.5
        load_util = node_data[1, 0] if node_data.shape[0] > 1 else 0.5
        priority = node_data[2, 0] if node_data.shape[0] > 2 else 0.5
        delay_prob = node_data[3, 0] if node_data.shape[0] > 3 else 0.1
        congestion = node_data[4, 0] if node_data.shape[0] > 4 else 0.2
        energy_level = node_data[5, 0] if node_data.shape[0] > 5 else 0.5

        # Normalize features for fitness calculation
        norm_travel = travel_time / (np.max(travel_time)+1e-8) if travel_time > 0 else 0.5
        norm_load = load_util / (np.max(load_util)+1e-8) if load_util > 0 else 0.5
        norm_priority = priority / (np.max(priority)+1e-8) if priority > 0 else 0.5
        norm_congestion = congestion / (np.max(congestion)+1e-8) if congestion > 0 else 0.2
        norm_energy = energy_level / (np.max(energy_level)+1e-8) if energy_level > 0 else 0.5

        # --- ACO-like metric function with improved fitness ---
        def aco_metric(base):
            pheromones = np.full(self.aco_num_ants, self.pheromone_init)
            best_value = base
            for _ in range(self.aco_iterations):
                # Candidate solutions influenced by pheromones + randomness
                candidates = base + pheromones + 0.1*np.random.randn(self.aco_num_ants)

                # Multi-objective fitness
                fitness = (
                    (1 - norm_travel) * 0.4 +
                    norm_load * 0.3 +
                    norm_priority * 0.2 +
                    (1 - norm_congestion) * 0.05 +
                    norm_energy * 0.05
                ) + 0.05*np.random.randn(self.aco_num_ants)

                # Update pheromones: evaporation + reinforcement
                pheromones = (1 - self.evaporation) * pheromones + self.evaporation * fitness

                # Keep track of best candidate
                best_value = candidates[np.argmax(fitness)]

            return best_value

        # Compute node metrics using improved ACO metric
        wait = 1 / (1 + aco_metric(travel_time * (1 + congestion*0.5)))
        throughput = aco_metric(load_util * np.sqrt(energy_level + 0.1))
        util = aco_metric(priority * (1 - delay_prob) + 0.1 * energy_level)
        patience = 1 / (1 + aco_metric(congestion * (1 + delay_prob)))

        score = wait + throughput + util + patience  # weighted sum to reduce over-unification

        return {
            'wait': wait,
            'throughput': throughput,
            'util': util,
            'patience': patience,
            'score': score
        }
candidate_dims = [[i] for i in candidate_dims]
candidate_dims

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


# ---------------- CONFIG ----------------nsion

inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_iters_per_outer = 50
outer_generations = 10
outer_cost_limit = 10000
inner_learning = 0.1
gamma_interlayer = 1
top_k = 263
seed = np.random.seed()

new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])



class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim[0] if isinstance(inter_dim, list) else (inter_dim if inter_dim is not None else max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim)
        self.max_input = 2 * (max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim)
                    # Initialize weights proportional to synthetic correlation between nodes
        self.weights = {}
        self.bias = {}
        for i in range(D_graph):
            for j in range(D_graph):
                if i != j:
                    # small random + slight bias towards correlation
                    w_init = np.random.uniform(-0.1, 0.1, (self.inter_dim, self.max_input))
                    self.weights[(i,j)] = w_init
                    self.bias[(i,j)] = np.zeros(self.inter_dim)

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0, self.max_input - len(concat))))[:self.max_input]

        # Normalize input to improve correlation
        concat = (concat - np.mean(concat)) / (np.std(concat) + 1e-12)

        # Compute activation
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]

        # Scale by correlation strength with input signals
        input_strength = np.clip(np.mean(np.abs(concat)), 0, 1)
        v = v * input_strength

        return 1 / (1 + np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i == j:
                    continue
                if abs(Gmat[i,j]) > self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i, j, nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts) < 2:
            return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1, keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2, axis=1) / (A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds, stds) + 1e-12)
        np.fill_diagonal(corr, 0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat, nested_reps)
        if not acts:
            return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

    def correlate_shrink_interlayer(self, fmt_bounds=None, interaction_tensor=None, metrics_keys=None, verbose=True):
        """
        Compute Pearson correlation per node & metric between:
            - shrink factor (adaptive FMT)
            - mean outgoing inter-layer activations
        Returns: {node_idx: {metric: {'r':..., 'p':...}}}
        """
        from scipy.stats import pearsonr

        if metrics_keys is None:
            metrics_keys = ['wait','throughput','util','patience']

        D = self.D_graph

        # 1. Compute FMT bounds if not given
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_bounds_adaptive(top_k=top_k)

        # 2. Get inter-layer activations if not provided
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)
        shrink_factors = self.compute_fmt_shrink_factor(fmt_bounds, metrics_keys)  # (D, num_metrics)

        correlations = {}

        for i in range(D):
            correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT shrink for node i (broadcasted across outgoing edges)
                shrink_vec = shrink_factors[i, k] * np.ones(D)
                # Outgoing inter-layer activations from node i
                inter_vec = inter_mean[i, :]
                # Remove self-loop
                mask = np.arange(D) != i
                shrink_vec = shrink_vec[mask]
                inter_vec = inter_vec[mask]

                # Compute Pearson correlation
                if np.std(inter_vec) > 1e-8:  # valid correlation
                    r, p = pearsonr(shrink_vec, inter_vec)
                else:
                    r, p = 0.0, 1.0  # no variability

                correlations[i][key] = {'r': r, 'p': p}
                if verbose:
                    print(f"Node {i} | {key} shrink vs inter-layer: r={r:.3f}, p={p:.3e}")

        return correlations



# ---------------- UNIFIED ACOR MULTIPLEX ----------------
class Fuzzy_Hierarchical_Multiplex:
    def __init__(self, candidate_dims, D_graph, inner_archive_size, inner_offspring,
                 outer_archive_size, outer_offspring, synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True,metrics=METRIC_KEYS):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.inner_archive_size = inner_archive_size
        self.inner_offspring = inner_offspring
        self.outer_archive_size = outer_archive_size
        self.outer_offspring = outer_offspring
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag
        self.best_dim_per_node = [len(t)-1 for t in synthetic_targets]  # last element as best dim

        self.MK = metrics
        self.MKI = metrics+['score']
        self.nested_reps = [np.zeros(c[0]) for c in candidate_dims]
      #  self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []

    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
              steps=100, lr_x=0.001, lr_y=0.001, lr_W=0.001,
              decorrelate_metrics=True):

        # --- Initialize activations ---
        x = target.copy()#np.random.uniform(0, 1, D_fcm)

        y = np.random.uniform(-0.6, 0.6, (D_fcm))

        # Pad target for L2 computation
        target_padded = np.pad(target, (0, len(self.nested_reps[node_idx]) - len(target)),
                            mode='constant', constant_values=0.5)
        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx] - target_padded))

        # --- FCM updates ---
        W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
        np.fill_diagonal(W, 0)

        for _ in range(steps):
            z = y.dot(W) + x
            Theta_grad_z = z - target
            Theta_grad_x = Theta_grad_z
            Theta_grad_y = Theta_grad_z.dot(W.T)
            Theta_grad_W = np.outer(y, Theta_grad_z)
         #   Theta_grad_z = 2 * z - target_padded
          #  Theta_grad_x = W.T @ Theta_grad_z + (y + 1)

           # Theta_grad_y = x + 1
            #Theta_grad_W = np.outer(Theta_grad_z, x)

            x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
            y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)
            W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)

            x = np.clip(x, 0, 1)
            y = np.clip(y, 0, 1)
            np.fill_diagonal(W, 0)
            W = np.clip(W, -1, 1)

        # Pad FCM output to max dim for nested_reps
        x_padded = np.pad(x, (0, len(self.nested_reps[node_idx]) - len(x)),
                        mode='constant', constant_values=0.5)
        self.nested_reps[node_idx] = x_padded
        self.l2_after.append(np.linalg.norm(x_padded - target_padded))

        # --- Compute metrics ONCE ---
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)
        metrics = metrics_evaluator.compute_node_metrics(node_idx, y=x)

        # --- Compute MI score for inter-layer ---
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return x, y, W, mi_score, metrics


    # ---------- OUTER LOOP ----------
    def run_outer(self):
        """
        Compute node contributions and outer score with normalized weights.
        Uses raw node metrics + FMT sums, plus correlation penalty.
        Weighted FMT and best_weights are stored as attributes.
        """
        node_metrics_list = self.capped_node_metrics
        raw_scores = np.array([m['score'] for m in node_metrics_list])
        D = self.D_graph

        # --- Compute Fuzzy Metric Tensor ---
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=False)  # no normalization
        gamma = self.inter_layer.gamma

        # --- Compute per-node fuzzy sums ---
        fmt_node_sums = np.array([
            fuzzy_tensor[i,:,:].sum() - fuzzy_tensor[i,i,:].sum()
            for i in range(D)
        ])

        # --- Prepare for optimization ---
        interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)
        fmt_mean = fuzzy_tensor.mean(axis=2)
        inter_mean = interaction_tensor.mean(axis=2)

        lambda_reg = 0.05

        def objective(weights):
            node_contrib = weights * (raw_scores + gamma * fmt_node_sums)

            # correlation penalty
            corr_penalty = 0.0
            for i in range(D):
                fmt_vec = fmt_mean[i,:]
                inter_vec = inter_mean[i,:]
                if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr_penalty += abs(np.corrcoef(fmt_vec, inter_vec)[0,1])**2
            corr_penalty /= D

            # regularization: keep weights near uniform
            reg_penalty = lambda_reg * np.sum((weights - 1.0/D)**2)

            return -(node_contrib.sum() - corr_penalty - reg_penalty)

        # bounds: weights positive
        bounds = [(0.2, 0.3) for _ in range(D)]
        # constraint: sum(weights) = 1
        cons = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}

        from scipy.optimize import minimize
        result = minimize(objective, x0=np.ones(D)/D, bounds=bounds, constraints=cons, method='SLSQP')
        best_weights = result.x

        # --- compute final contributions ---
        node_contributions = best_weights * (raw_scores + gamma * fmt_node_sums)

        # --- weighted FMT ---
        weighted_fmt = fuzzy_tensor.copy()
        for i in range(D):
            weighted_fmt[i,:,:] *= best_weights[i]  # scale row by node weight

        # --- compute correlation penalty separately ---
        corr_penalty = 0.0
        for i in range(D):
            fmt_vec = fmt_mean[i,:]
            inter_vec = inter_mean[i,:]
            if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
                corr_penalty += abs(np.corrcoef(fmt_vec, inter_vec)[0,1])**2
        corr_penalty /= D

        combined_score = node_contributions.sum() - corr_penalty

        # Save attributes for later use
        self.node_score_contributions = node_contributions
        self.correlation_penalty = corr_penalty
        self.best_weights = best_weights
        self.weighted_fmt = weighted_fmt

        return node_metrics_list, combined_score, node_contributions




    def run(self, outer_generations=outer_generations):
        final_metrics = None

        for gen in range(outer_generations):
            mi_scores = []
            node_metrics_list = []

            for node_idx in range(self.D_graph):
                # Full target vector for this node
                full_target = self.synthetic_targets[node_idx]['target']

                # Use the candidate dimension assigned to this node
                D_fcm = self.candidate_dims[node_idx][0]  # get the integer
                target = full_target[:D_fcm]


                # Run inner FCM
                _, _, _, mi_score, metrics = self.run_inner(node_idx, target, D_fcm)
                mi_scores.append(mi_score)
                node_metrics_list.append(metrics)


            # --- Outer loop uses decorrelated metrics ---
            self.capped_node_metrics = node_metrics_list
            _, capped_score, node_contributions = self.run_outer()  # uses self.capped_node_metrics

            print(f"\n--- Generation {gen} Metrics ---")
            for i, m in enumerate(node_metrics_list):
                print(f"Node {i} | " + " | ".join([f"{k}: {v:.2f}" for k,v in m.items()]))

            print(f"\n--- Generation {gen} Node Contributions ---")
            for i, c in enumerate(node_contributions):
                print(f"Node {i}: Contribution = {c:.4f}")

            print(f"Outer Score (capped): {capped_score:.3f}")

            final_metrics = node_metrics_list

        return final_metrics


    # ---------- VISUALIZATIONS ----------
    # ---------- VISUALIZATIONS ----------
    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14,3))
        for i in range(self.D_graph):
            # Node's actual dimension
            dim_i = self.candidate_dims[i][0]  # ✅ integer
            base = self.nested_reps[i][:dim_i]  # slice to candidate dim
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base

            # True target for this node, sliced to candidate dim
            y_true = self.synthetic_targets[i]['target'][:len(y_sel)]
            if len(y_true) < len(y_sel):
                y_true = np.pad(y_true, (0, len(y_sel)-len(y_true)), "constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1,self.D_graph,i+1)
            plt.fill_between(range(len(y_min)),y_min,y_max,color='skyblue',alpha=0.4,label='Elite Interval')
            plt.plot(y_sel,'k-',lw=2,label='Estimated')
            plt.plot(y_true,'r--',lw=2,label='True')
            plt.ylim(0,1.05)
            plt.title(f"Node {i+1}")
            if i==0: plt.legend()
        plt.tight_layout()
        plt.show()


    def plot_nested_activations(self):
        plt.figure(figsize=(12,3))
        for i,rep in enumerate(self.nested_reps):
            dim_i = self.candidate_dims[i][0]
            rep_i = rep[:dim_i]  # slice to candidate dim
            plt.subplot(1,self.D_graph,i+1)
            plt.bar(range(len(rep_i)), rep_i, color=plt.cm.plasma(rep_i))
            plt.ylim(0,1)
            plt.title(f"Node {i+1}")
        plt.tight_layout()
        plt.show()


    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i!=j and abs(self.chosen_Gmat[i,j])>0.02:
                    G.add_edge(i,j,weight=self.chosen_Gmat[i,j])
        node_sizes = [self.best_dim_per_node[i]*200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight']>0 else 'red' for _,_,d in G.edges(data=True)]
        edge_widths = [abs(d['weight'])*3 for _,_,d in G.edges(data=True)]
        pos = nx.circular_layout(G)
        plt.figure(figsize=(6,6))
        nx.draw(G,pos,node_size=node_sizes,node_color='skyblue',
                edge_color=edge_colors,width=edge_widths,arrows=True,with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()
# ---------------- INTERACTIONS INSPECTOR ----------------

    def print_interactions(self, return_tensor=True, verbose=True):
            D_graph = self.D_graph
            inter_dim = self.inter_layer.inter_dim
            inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

            acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
            if not acts:
                if verbose:
                    print("No active edges above threshold.")
                return inter_tensor if return_tensor else None

            for (i, j), vec in acts.items():
                inter_tensor[i, j, :] = vec
                if verbose:
                    act_str = ", ".join([f"{v:.3f}" for v in vec])
                    print(f"Node {i} -> Node {j}: [{act_str}]")
            return inter_tensor if return_tensor else None

        # Move these outside of print_interactions (class-level)
    def print_l2_summary(self):
            print("\nL2 Distances to Target per Node:")
            for idx, (before, after) in enumerate(zip(self.l2_before, self.l2_after)):
                print(f"Node {idx}: Before={before:.4f}, After={after:.4f}")

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            """
            Computes a Fuzzy Metric Tensor (D_graph x D_graph x num_metrics)
            using current nested reps and node metrics.
            Each slice [i,j,:] represents metrics of node j (optionally weighted by Gmat[i,j])
            """
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i==j:
                        tensor[i,j,:] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                        tensor[i,j,:] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor



    def compute_fmt_shrink_factor(self, fmt_bounds, metrics_keys=None):
        """
        Returns shrink factor per node and metric.
        shrink_factor = 1 - (current_interval / original_interval)
        """
        if metrics_keys is None:
            metrics_keys = ['wait','throughput','util','patience']

        D = self.D_graph
        num_metrics = len(metrics_keys)
        shrink_factors = np.zeros((D, num_metrics))

        for i in range(D):
            for k in range(num_metrics):
                lower, upper = fmt_bounds[i, i, k, 0], fmt_bounds[i, i, k, 1]  # self-node interval
                interval_width = upper - lower + 1e-12  # normalized [0,1]
                shrink_factors[i, k] = 1 - interval_width  # more shrink = higher value

        return shrink_factors

    def compute_fmt_with_bounds_adaptive(self, top_k=21, max_shrink=0.5, metrics_keys=None):
        """
        Computes FMT bounds using pointwise min/max across elite solutions,
        and applies dynamic adaptive shrinking where variability is low.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        if metrics_keys is None:
            metrics_keys = ['wait','throughput','util','patience']

        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D, D, num_metrics, 2))
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        variability = np.zeros((D, num_metrics))

        # Step 1: compute bounds from perturbations
        for i in range(D):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx, :] = [m[k] for k in metrics_keys]

            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)
            tensor_bounds[i, :, :, 0] = lower_i[np.newaxis, :]  # broadcast to all j
            tensor_bounds[i, :, :, 1] = upper_i[np.newaxis, :]
            variability[i, :] = metrics_matrix.std(axis=0)

        # Step 2: adaptive shrinking
        for i in range(D):
            for j in range(D):
                for k in range(num_metrics):
                    lower, upper = tensor_bounds[i,j,k,0], tensor_bounds[i,j,k,1]
                    mean = (lower + upper)/2
                    var_norm = min(1.0, variability[i,k]/(upper-lower + 1e-12))
                    shrink_factor = max_shrink * (1 - var_norm)
                    tensor_bounds[i,j,k,0] = mean - shrink_factor*(mean - lower)
                    tensor_bounds[i,j,k,1] = mean + shrink_factor*(upper - mean)

        return tensor_bounds



    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot a heatmap panel for each metric in the FMT.
        Rows: source node i
        Columns: target node j
        """
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:,:,k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j,i,f"{data[i,j]:.2f}",ha='center',va='center',color='white',fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        """
        Computes FMT bounds using pointwise min/max across elite solutions.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        metrics_keys = ['wait','throughput','util','patience']
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D,D,num_metrics,2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        for i in range(D):
            # Generate top_k perturbations around current nested_rep (like in plot_pointwise_minmax_elite)
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)

            # Compute node metrics for each perturbed solution
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx,:] = [m[k] for k in metrics_keys]

            # Compute pointwise min/max across elite solutions
            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            # Fill bounds tensor for all source nodes (i->j)
            for j in range(D):
                tensor_bounds[i,j,:,0] = lower_i
                tensor_bounds[i,j,:,1] = upper_i

        return tensor_bounds


    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        D = self.D_graph
        metrics_keys = self.MK

        # Extract lower/upper for each metric and compute mean
        mean_vals = (fmt_tensor_bounds[:, :, :, 0] + fmt_tensor_bounds[:, :, :, 1]) / 2  # (D,D,M_actual)
        M_actual = mean_vals.shape[2]

        # Apply outer-loop weights if available
        if hasattr(self, 'best_weights'):
            for i in range(D):
                mean_vals[i, :, :] *= self.best_weights[i]

        # Take any column (say j=0) for unique row values
        unique_mean = mean_vals[:, 0, :]  # shape (D, M_actual)

        # Plotting
        fig, ax = plt.subplots(figsize=(1.2*M_actual + 4, 0.35*D + 4))
        im = ax.imshow(unique_mean, cmap='viridis', aspect='auto', vmin=0, vmax=unique_mean.max())

        # Annotate heatmap with weighted mean values
        for i in range(D):
            for k in range(M_actual):
                ax.text(k, i, f"{unique_mean[i,k]:.2f}", ha='center', va='center', color='white', fontsize=8)

        # Label axes
        ax.set_xticks(range(M_actual))
        ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
        ax.set_yticks(range(D))
        ax.set_yticklabels([f"Node {i}" for i in range(D)])

        ax.set_title("Weighted FMT (Outer Loop Weights)")
        fig.colorbar(im, ax=ax, label='Weighted Mean Metric Value')
        plt.tight_layout()
        plt.show()

    def plot_node_score_contribution(self, metrics_keys=METRIC_KEYS):
        """
        Plot per-node total score contribution:
            - 3 panels: Raw, FMT (interaction), Total
            - Diagonal represents raw contributions
            - FMT scaled by best_weights if available
            - Annotated cells
            - Normalized across metrics for consistent visualization
        """
        D = self.D_graph
        node_contributions = np.array(self.node_score_contributions)

        # --- FMT contribution ---
        if hasattr(self, 'weighted_fmt'):
            fuzzy_tensor = np.array(self.weighted_fmt)
        else:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        # Normalize across entire tensor for plotting
        fuzzy_tensor_norm = (fuzzy_tensor - fuzzy_tensor.min()) / (fuzzy_tensor.max() - fuzzy_tensor.min() + 1e-12)
        fmt_matrix = fuzzy_tensor_norm.sum(axis=2)  # sum over metrics
        np.fill_diagonal(fmt_matrix, 0)

        # --- Raw matrix on diagonal ---
        raw_matrix = np.zeros((D,D))
        np.fill_diagonal(raw_matrix, node_contributions)

        # --- Total contribution ---
        total_matrix = raw_matrix + fmt_matrix

        # --- Global min/max for color scale ---
        global_min, global_max = total_matrix.min(), total_matrix.max()

        # --- Plot ---
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "Normalized FMT Contribution", "Total Contribution"]

        for ax, mat, title in zip(axes, matrices, titles):
            im = ax.imshow(mat, cmap='viridis', vmin=0, vmax=1)  # normalized
            for i in range(D):
                for j in range(D):
                    ax.text(j, i, f"{mat[i,j]:.2f}", ha='center', va='center', color='white', fontsize=8)
            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i+1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i+1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Contribution Value')
        plt.tight_layout()
        plt.show()

    def correlate_fmt_interactions_per_node(self, fmt_bounds=None, interaction_tensor=None, verbose=True):
        """
        Correlate the FMT bounds with inter-layer interactions per node and per metric.
        Returns a dict of shape: {node_idx: {metric: {'r':..., 'p':...}}}.
        """
        from scipy.stats import pearsonr
        import matplotlib.pyplot as plt

        metrics_keys = ['wait','throughput','util','patience']
        D = self.D_graph

        # Compute tensors if not provided
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=21)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        # Reduce interaction tensor along inter_dim
        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)

        node_correlations = {}

        for i in range(D):
            node_correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT bounds for target node j from source i (mean of lower/upper)
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)  # shape (D,)
                # Interaction tensor for edges from node i to j
                inter_vec = inter_mean[i,:]  # shape (D,)
                # Pearson correlation
                corr, pval = pearsonr(fmt_mean, inter_vec)
                node_correlations[i][key] = {'r': corr, 'p': pval}

                if verbose:
                    print(f"Node {i} | {key}: r = {corr:.3f}, p = {pval:.3e}")
                    plt.figure(figsize=(4,3))
                    plt.scatter(fmt_mean, inter_vec, alpha=0.7, edgecolor='k', color='skyblue')
                    plt.xlabel(f"FMT {key} (Node {i} -> others)")
                    plt.ylabel(f"Interaction mean (Node {i} -> others)")
                    plt.title(f"Node {i} | {key} correlation: r={corr:.3f}")
                    plt.grid(True)
                    plt.show()

        return node_correlations

    def correlation_penalty(self, fmt_bounds=None, interaction_tensor=None):
        """
        Computes a penalty term that is high if per-node FMT metrics correlate with interactions.
        Returns total penalty to subtract from the outer score.
        """
        from scipy.stats import pearsonr

        D = self.D_graph
        metrics_keys = ['wait','throughput','util','patience']

        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=top_k)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)
        total_penalty = 0.0

        for i in range(D):
            for k in range(len(metrics_keys)):
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)
                inter_vec = inter_mean[i,:]
                if np.std(fmt_mean) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr, _ = pearsonr(fmt_mean, inter_vec)
                    total_penalty += abs(corr)  # penalize high correlation

        # normalize by number of nodes × metrics
        total_penalty /= (D * len(metrics_keys))**2
        return total_penalty


# ---------------- USAGE ----------------
if __name__ == "__main__":
    optimizer = Fuzzy_Hierarchical_Multiplex(
        candidate_dims, D_graph,
        inner_archive_size, inner_offspring,
        outer_archive_size, outer_offspring,
        synthetic_targets,
        inner_learning, gamma_interlayer=1,
        causal_flag=False
    )
    metrics_list = optimizer.run()
    optimizer.plot_pointwise_minmax_elite()
    optimizer.plot_nested_activations()
    # Compute FMT with elite bounds
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k+10)

# Plot as heatmaps
    optimizer.plot_fmt_with_bounds(fmt_elite_bounds)

    # Compute fuzzy multiplex tensor
    fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=True)
    optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

    # Compute FMT with bounds (minimax elite intervals)
    optimizer.plot_node_score_contribution()
    optimizer.plot_outer_fuzzy_graph()
  #  optimizer.print_interactions()
    tensor = optimizer.print_interactions()

    print("Tensor shape:", tensor.shape,'\n',tensor)
    # Compute tensors first
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k)
    interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)


    interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

    # Get per-node, per-metric correlations
    #node_metric_corrs = optimizer.correlate_fmt_interactions_per_node(
     #   fmt_bounds=fmt_elite_bounds,
      #  interaction_tensor=interaction_tensor
   # )

In [ ]:

import numpy as np

class GDFCMPredictorForward:
    """
    Forward predictor for GDFCM.
    Can handle new input data (DATA_MATRIX) and compute:
        - Node activations
        - Node metrics
        - Inter-layer MI scores
        - Total contributions and outer score
    """
    def __init__(self, trained_gdfcm: Fuzzy_Hierarchical_Multiplex):
        self.gdfcm = trained_gdfcm
        self.D_graph = trained_gdfcm.D_graph
        self.nested_reps = trained_gdfcm.nested_reps
        self.chosen_Gmat = trained_gdfcm.chosen_Gmat
        self.inter_layer = trained_gdfcm.inter_layer

    def predict_node(self, node_idx, data_row):
        """
        Predict metrics and activations for a single node given a new data row.
        """
        # Use stored nested_rep as starting point
        x = self.nested_reps[node_idx].copy()
        # Optional: you could combine with data_row to modify x
        # For now, we just compute metrics using data_row as the input
        metrics_evaluator = MetricsEvaluator(np.array([data_row]))
        metrics = metrics_evaluator.compute_node_metrics(0, y=x)

        # Inter-layer MI
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return {'activations': x, 'metrics': metrics, 'mi_score': mi_score}

    def predict_all_nodes(self, new_data_matrix):
        """
        Predict metrics and activations for all nodes using new data.
        new_data_matrix: shape (D_graph, num_features)
        """
        results = []
        for node_idx in range(self.D_graph):
            data_row = new_data_matrix[node_idx % len(new_data_matrix)]
            node_result = self.predict_node(node_idx, data_row)
            results.append(node_result)
        return results

    def predict_scores(self):
        """
        Compute per-node contributions and total outer score.
        """
        _, total_score, node_contributions = self.gdfcm.run_outer()
        return {'node_contributions': node_contributions, 'total_score': total_score}
# Suppose optimizer is your trained GDFCM
predictor = GDFCMPredictorForward(optimizer)

# New data: same number of nodes, each with same num_features

# Predict all nodes
predictions = predictor.predict_all_nodes(new_DATA_MATRIX)

for idx, node_pred in enumerate(predictions):
    print(f"Node {idx} Metrics:", node_pred['metrics'])
    print(f"Node {idx} Activations:", node_pred['activations'])
    print(f"Node {idx} MI Score:", node_pred['mi_score'])

# Optional: compute total contributions
score_info = predictor.predict_scores()
print("Node Contributions:", score_info['node_contributions'])
print("Total Score:", score_info['total_score'])

class GDFCMPredictorAdaptive:
    """
    Adaptive forward predictor for GDFCM.
    - Accepts new data per node.
    - Updates activations slightly using a mini inner-loop.
    - Computes metrics, inter-layer MI, and outer score contributions.
    """
    def __init__(self, trained_gdfcm: Fuzzy_Hierarchical_Multiplex, lr_x=0.01, lr_steps=10):
        self.gdfcm = trained_gdfcm
        self.D_graph = trained_gdfcm.D_graph
        self.nested_reps = [rep.copy() for rep in trained_gdfcm.nested_reps]
        self.chosen_Gmat = trained_gdfcm.chosen_Gmat
        self.inter_layer = trained_gdfcm.inter_layer
        self.lr_x = lr_x
        self.lr_steps = lr_steps

    def adapt_node(self, node_idx, data_row):
        """
        Mini inner-loop: slightly update activations based on new data row.
        Handles shape mismatch by projecting new data to node activation dim.
        """
        x = self.nested_reps[node_idx].copy()
        dim = len(x)

        # Simple linear projection of new data to activation space
        if len(data_row) != dim:
            # Use mean pooling to reduce or slice if too large
            if len(data_row) > dim:
                target = data_row[:dim]
            else:
                # Pad with 0.5
                target = np.pad(data_row, (0, dim - len(data_row)), 'constant', constant_values=0.5)
        else:
            target = data_row

        target = np.clip(target, 0, 1)

        # Mini inner-loop update
        for _ in range(self.lr_steps):
            grad = x - target
            x -= self.lr_x * grad
            x = np.clip(x, 0, 1)

        self.nested_reps[node_idx] = x

        # Compute metrics
        metrics_evaluator = MetricsEvaluator(np.array([data_row]))
        metrics = metrics_evaluator.compute_node_metrics(0, y=x)

        # Inter-layer MI
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return {'activations': x, 'metrics': metrics, 'mi_score': mi_score}


    def adapt_all_nodes(self, new_data_matrix):
        """
        Update activations for all nodes given new data matrix.
        """
        results = []
        for node_idx in range(self.D_graph):
            data_row = new_data_matrix[node_idx % len(new_data_matrix)]
            node_result = self.adapt_node(node_idx, data_row)
            results.append(node_result)
        return results

    def compute_outer_score(self):
        """
        Compute node contributions and total outer score using current nested_reps.
        """
        _, total_score, node_contributions = self.gdfcm.run_outer()
        return {'node_contributions': node_contributions, 'total_score': total_score}
# Initialize adaptive predictor
adaptive_predictor = GDFCMPredictorAdaptive(optimizer, lr_x=0.02, lr_steps=15)

# New data

# Adapt activations to new data
predictions = adaptive_predictor.adapt_all_nodes(new_DATA_MATRIX)

for idx, node_pred in enumerate(predictions):
    print(f"Node {idx} Metrics:", node_pred['metrics'])
    print(f"Node {idx} Activations:", node_pred['activations'])
    print(f"Node {idx} MI Score:", node_pred['mi_score'])

# Compute outer node contributions after adaptation
score_info = adaptive_predictor.compute_outer_score()
print("Node Contributions:", score_info['node_contributions'])
print("Total Score:", score_info['total_score'])
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error

pred_metrics = []
true_metrics = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    pred = adaptive_predictor.adapt_node(idx, data_row)
    pred_metrics.append([pred['metrics'][k] for k in ['wait','throughput','util','patience']])
    true_metrics.append([MetricsEvaluator(new_DATA_MATRIX).compute_node_metrics(idx)[k]
                         for k in ['wait','throughput','util','patience']])

pred_metrics = np.array(pred_metrics)
true_metrics = np.array(true_metrics)


def evaluate_predictions(true_metrics, pred_metrics, metric_names=None):
    """
    Evaluate multi-metric predictions per node and overall.

    Args:
        true_metrics: np.array, shape (num_nodes, num_metrics)
        pred_metrics: np.array, same shape as true_metrics
        metric_names: list of metric names (optional)

    Returns:
        dict with evaluation metrics
    """
    true_metrics = np.array(true_metrics)

    pred_metrics = np.array(pred_metrics)

    if metric_names is None:
        metric_names = [f'Metric {i}' for i in range(true_metrics.shape[1])]

    num_nodes, num_metrics = true_metrics.shape

    results = {'per_metric': {}, 'overall': {}}

    # Per metric
    for i, name in enumerate(metric_names):
        t = true_metrics[:, i]
        p = pred_metrics[:, i]
        mae = mean_absolute_error(t, p)
        rmse = np.sqrt(mean_squared_error(t, p))
        r2 = r2_score(t, p)
        corr = np.corrcoef(t, p)[0,1]
        mape = np.mean(np.abs((t - p) / (t + 1e-12))) * 100
        results['per_metric'][name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'Pearson': corr,
            'MAPE (%)': mape
        }

    # Overall
    mae_overall = mean_absolute_error(true_metrics.flatten(), pred_metrics.flatten())
    rmse_overall = np.sqrt(mean_squared_error(true_metrics.flatten(), pred_metrics.flatten()))
    r2_overall = r2_score(true_metrics.flatten(), pred_metrics.flatten())

    # Cosine similarity (averaged over nodes)
    cos_sim = np.mean([cosine_similarity(t.reshape(1,-1), p.reshape(1,-1))[0,0]
                       for t,p in zip(true_metrics, pred_metrics)])

    results['overall'] = {
        'MAE': mae_overall,
        'RMSE': rmse_overall,
        'R2': r2_overall,
        'CosineSim': cos_sim
    }

    return results
# Suppose true_metrics and pred_metrics have shape (D_graph, 4)
metrics_eval = evaluate_predictions(true_metrics, pred_metrics, metric_names=['wait','throughput','util','patience'])

# Print per-metric evaluation
for metric, vals in metrics_eval['per_metric'].items():
    print(f"{metric}: {vals}")

# Print overall evaluation
print("\nOverall metrics:", metrics_eval['overall'])




# Nested FCM

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ================================================================
#  SIMPLE FCM MODEL
# ================================================================
class SimpleFCM:
    def __init__(self, n_nodes, lr=0.05, max_iter=200, seed=0):
        self.n_nodes = n_nodes
        self.lr = lr
        self.max_iter = max_iter
        self.rng = np.random.default_rng(seed)
        self.W = self.rng.normal(0, 0.3, size=(n_nodes, n_nodes))
        np.fill_diagonal(self.W, 0.0)

    @staticmethod
    def activation(x):
        return 1.0 / (1.0 + np.exp(-x))

    def propagate(self, state):
        return self.activation(state @ self.W.T)

    def fit(self, X, Y):
        for _ in range(self.max_iter):
            idx = self.rng.integers(0, len(X))
            x = X[idx]
            y_true = Y[idx]
            y_pred = self.propagate(x)
            error = (y_true - y_pred)
            deltaW = self.lr * np.outer(error, x)
            np.fill_diagonal(deltaW, 0.0)
            self.W += deltaW

    def predict(self, X):
        return self.activation(X @ self.W.T)

# ================================================================
#  METRIC COMPUTATION: WAIT, THROUGHPUT, UTILIZATION, PATIENCE
# ================================================================
def compute_metrics(raw_data):
    """
    raw_data: (T, D, F) raw inputs for each node
    Returns: (T, D, 4) metrics [wait, throughput, utilization, patience]
    """
    T, D, F = raw_data.shape
    metrics = np.zeros((T, D, 4))  # wait, throughput, utilization, patience

    for t in range(T):
        for d in range(D):
            node_data = raw_data[t, d, :]

            # ---------------- CUSTOM FORMULAS ----------------
            # Replace these with your actual equations
            wait = node_data[0]            # e.g., avg queue length / arrival rate
            throughput = node_data[1]      # e.g., processed jobs / time interval
            utilization = node_data[2]     # e.g., busy time / total time
            patience = node_data[3]        # e.g., time until abandonment

            metrics[t, d, 0] = wait
            metrics[t, d, 1] = throughput
            metrics[t, d, 2] = utilization
            metrics[t, d, 3] = patience

    return metrics

# ================================================================
#  EVALUATION FUNCTION
# ================================================================
def evaluate_predictions(true_metrics, pred_metrics, metric_names=None):
    true_metrics = np.asarray(true_metrics)
    pred_metrics = np.asarray(pred_metrics)
    results = {'per_metric': {}, 'overall': {}}

    if true_metrics.ndim == 1:
        true_metrics = true_metrics[:, None]
    if pred_metrics.ndim == 1:
        pred_metrics = pred_metrics[:, None]

    num_nodes = true_metrics.shape[1]
    if metric_names is None:
        metric_names = [f'Node {i}' for i in range(num_nodes)]

    for i, name in enumerate(metric_names):
        t = true_metrics[:, i]
        p = pred_metrics[:, i]
        mae = mean_absolute_error(t, p)
        rmse = np.sqrt(mean_squared_error(t, p))
        r2 = r2_score(t, p)
        corr = np.corrcoef(t, p)[0, 1] if np.std(t) * np.std(p) > 0 else np.nan
        mape = np.mean(np.abs((t - p) / (t + 1e-12))) * 100
        results['per_metric'][name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'Pearson': corr, 'MAPE (%)': mape}

    mae_overall = mean_absolute_error(true_metrics.flatten(), pred_metrics.flatten())
    rmse_overall = np.sqrt(mean_squared_error(true_metrics.flatten(), pred_metrics.flatten()))
    r2_overall = r2_score(true_metrics.flatten(), pred_metrics.flatten())
    cos_sim = np.mean([cosine_similarity(t.reshape(1, -1), p.reshape(1, -1))[0, 0]
                       for t, p in zip(true_metrics, pred_metrics)])
    results['overall'] = {'MAE': mae_overall, 'RMSE': rmse_overall, 'R2': r2_overall, 'CosineSim': cos_sim}

    return results

# ================================================================
#  FCM PIPELINE
# ================================================================
def run_fcm_pipeline(micro_data, test_size=0.2, lr=0.05, max_iter=500):
    T, D, M = micro_data.shape
    N = D * M
    # Flatten and normalize
    DATA_MATRIX = micro_data.reshape(T, N)
    DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0) + 1e-8)

    X_train, X_test = train_test_split(DATA_MATRIX, test_size=test_size, random_state=42)
    y_train, y_test = X_train.copy(), X_test.copy()

    fcm = SimpleFCM(n_nodes=N, lr=lr, max_iter=max_iter)
    fcm.fit(X_train, y_train)
    y_pred = fcm.predict(X_test)

    metric_names = [f"N{n}-M{m}" for n in range(D) for m in range(M)]
    metrics_eval = evaluate_predictions(y_test, y_pred, metric_names)
    return fcm, metrics_eval

# ================================================================
#  TWO-LAYER NESTED FCM PIPELINE
# ================================================================
def run_nested_fcm_pipeline(raw_data, test_size=0.2, lr=0.05, max_iter_layer1=500, max_iter_layer2=300):
    """
    raw_data: (T, D, F) raw inputs per node
    Layer 1: compute wait/throughput/util/patience and FCM
    Layer 2: inter-node FCM on Layer 1 predictions
    """
    # Compute Layer 1 metrics
    micro_data_metrics = compute_metrics(raw_data)

    # Layer 1 FCM
    fcm_layer1, eval_layer1 = run_fcm_pipeline(micro_data_metrics, test_size=test_size,
                                               lr=lr, max_iter=max_iter_layer1)

    # Get Layer 1 predictions
    T, D, M = micro_data_metrics.shape
    X_flat = micro_data_metrics.reshape(T, D * M)
    layer1_preds = fcm_layer1.predict(X_flat)
    layer1_preds_norm = (layer1_preds - layer1_preds.min(axis=0)) / (np.ptp(layer1_preds, axis=0) + 1e-8)
    layer1_preds_reshaped = layer1_preds_norm.reshape(T, D, M)

    # Layer 2 FCM on flattened Layer 1 outputs
    fcm_layer2, eval_layer2 = run_fcm_pipeline(layer1_preds_reshaped, test_size=test_size,
                                               lr=lr, max_iter=max_iter_layer2)

    return {
        'fcm_layer1': fcm_layer1,
        'eval_layer1': eval_layer1,
        'fcm_layer2': fcm_layer2,
        'eval_layer2': eval_layer2,
        'layer1_preds': layer1_preds_reshaped
    }

# ================================================================
#  Example Usage
# ================================================================
# raw_micro_data shape: (T, D, F)
# Example: T=100, D=5 nodes, F=4 raw inputs per node
T, D, F = 100, 5, 4
#raw_micro_data = np.random.rand(T, D, F)  # replace with actual raw data
# Suppose new_DATA_MATRIX shape is (T, D*F)
T = DATA_MATRIX.shape[0]        # number of samples
D = 5                               # number of nodes
F = 4                               # metrics per node
T, total_features = DATA_MATRIX.shape
D, F = 5, 4  # number of nodes and features per node

# Create a separate copy for experiments
new_DATA_MATRIX = np.random.rand(T, D * F)  # or any transformed version of DATA_MATRIX

# Convert to raw_micro_data format for pipelines
raw_micro_data = new_DATA_MATRIX[:, :D*F].reshape(T, D, F)

# Now call the nested FCM pipeline
nested_results = run_nested_fcm_pipeline(raw_micro_data)


print("\n--- Layer 1 Overall Metrics ---")
print(nested_results['eval_layer1']['overall'])

print("\n--- Layer 2 Overall Metrics ---")
print(nested_results['eval_layer2']['overall'])


# 2o fold cross val plus plots


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ================================
# Prepare the data
# ================================
# Each list corresponds to successive runs
overall = [
    {'MAE': 0.21564946873304658, 'RMSE': 0.26470920737196973, 'R2': -0.00785488396420786, 'CosineSim': 0.9506788398485098},
]

layer1 = [
    {'MAE': 0.47950894885825274, 'RMSE': 0.54544538971873, 'R2': -0.6321602784216089, 'CosineSim': 0.5926077084069774},
]

layer2 = [
    {'MAE': 0.21058072295411492, 'RMSE': 0.26161556079489284, 'R2': -0.31788372861360226, 'CosineSim': 0.8683080591282449},
]
# Append new data
# ================================
# Append new data
# ================================

# Overall metrics
overall += [
    {'MAE': 0.1629827926682889, 'RMSE': 0.20426620872184137, 'R2': 0.16141449909148786, 'CosineSim': 0.9774497096398342},
    {'MAE': 0.1494274469017904, 'RMSE': 0.1916450824776249, 'R2': 0.5754891507896205, 'CosineSim': 0.972403385741368},
    {'MAE': 0.14477234641326112, 'RMSE': 0.19886745884970572, 'R2': 0.14645965496213442, 'CosineSim': 0.9898544865985734},
    {'MAE': 0.15083137502333846, 'RMSE': 0.20069313516301185, 'R2': 0.31514229954682893, 'CosineSim': 0.9909085160776684},
    {'MAE': 0.16698794677672282, 'RMSE': 0.2255291695086502, 'R2': 0.46764564110737583, 'CosineSim': 0.9665007380349483},
    {'MAE': 0.1274361249685806, 'RMSE': 0.15800213580328085, 'R2': 0.6330454425987863, 'CosineSim': 0.9827155979359261},
    {'MAE': 0.15562711704292514, 'RMSE': 0.1909811795443274, 'R2': 0.6146197479044794, 'CosineSim': 0.975495383790852},
    {'MAE': 0.19801581829936352, 'RMSE': 0.2698410337331254, 'R2': 0.35575874309407696, 'CosineSim': 0.9760910997268285}
]

# Layer 1 metrics
layer1 += [
    {'MAE': 0.46495849269172024, 'RMSE': 0.5184341311193555, 'R2': -1.9944592742964478, 'CosineSim': 0.595690228107242},
    {'MAE': 0.5013428422208395, 'RMSE': 0.5640831352706341, 'R2': -1.2712502958200451, 'CosineSim': 0.6365558237107483},
    {'MAE': 0.4469742920929646, 'RMSE': 0.5315516786088597, 'R2': -0.956581661099934, 'CosineSim': 0.6715599013276247},
    {'MAE': 0.5044368739697064, 'RMSE': 0.5666259052985094, 'R2': -1.4925206922384935, 'CosineSim': 0.5687642030891207},
    {'MAE': 0.41712628431908294, 'RMSE': 0.5015717831725728, 'R2': -0.4309046922925137, 'CosineSim': 0.6912249451982717},
    {'MAE': 0.3822593900070547, 'RMSE': 0.45872410879482844, 'R2': -0.6933259879238522, 'CosineSim': 0.700987204387361},
    {'MAE': 0.5106473056094253, 'RMSE': 0.5781218248217179, 'R2': -0.8119768207595237, 'CosineSim': 0.5704982590814822},
    {'MAE': 0.5665484120191906, 'RMSE': 0.6142788021787405, 'R2': -1.3452800471365034, 'CosineSim': 0.601204223893042}
]

# Layer 2 metrics
layer2 += [
    {'MAE': 0.12144601623424855, 'RMSE': 0.1574874551746722, 'R2': -0.33838593381741844, 'CosineSim': 0.9648440861102592},
    {'MAE': 0.17975243718916076, 'RMSE': 0.23437529141222443, 'R2': 0.4521118378519421, 'CosineSim': 0.9140551604252312},
    {'MAE': 0.15019182014637725, 'RMSE': 0.22882669412019147, 'R2': 0.576331301764663, 'CosineSim': 0.9502856835156196},
    {'MAE': 0.24335298718120776, 'RMSE': 0.295109732683282, 'R2': 0.0020545294545663007, 'CosineSim': 0.8395885858768151},
    {'MAE': 0.15676719897136732, 'RMSE': 0.22105046223490232, 'R2': 0.3363948979076835, 'CosineSim': 0.9369767101270344},
    {'MAE': 0.1390374538291917, 'RMSE': 0.17168398445248786, 'R2': 0.5458515574244223, 'CosineSim': 0.9562047363881557},
    {'MAE': 0.14581237976681488, 'RMSE': 0.18986678235052232, 'R2': 0.4377844663038466, 'CosineSim': 0.9440387910675837},
    {'MAE': 0.15795282066248065, 'RMSE': 0.1938738593610289, 'R2': 0.4629110927122362, 'CosineSim': 0.9504218895995449}
]


# ================================
# Extract metrics for plotting
# ================================
runs = np.arange(1, len(overall)+1)

def extract_metric(data, key):
    return [d[key] for d in data]

metrics_list = ['MAE', 'RMSE', 'R2', 'CosineSim']

plt.figure(figsize=(20
                    , 10))

for i, metric in enumerate(metrics_list, 1):
    plt.subplot(2, 2, i)
    plt.plot(runs, extract_metric(overall, metric), '-o', label='Overall')
    plt.plot(runs, extract_metric(layer1, metric), '-o', label='FCM with one Layer')
    plt.plot(runs, extract_metric(layer2, metric), '-o', label='FCM with nested Layers, Not multiplexed')
    plt.title(metric)
    plt.xlabel('Run')
    plt.ylabel(metric)
    plt.grid(True)
    plt.legend()

plt.tight_layout()
plt.show()
